## 데이터 분석 전 기본적인 환경 setting

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/olist

/content/drive/MyDrive/olist


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
files = [f for f in os.listdir() if f.endswith(".csv")]
print(files)

['olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_items_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_orders_dataset.csv', 'olist_products_dataset.csv', 'product_category_name_translation.csv', 'olist_sellers_dataset.csv', 'all_dataset2.csv', 'all_dataset3.csv', 'trpicaol.csv', 'tropical.csv', 'tropical_state.csv', 'dataset.csv', 'second_data.csv', 'olist_order_reviews_dataset.csv']


In [ ]:
# order
orders = pd.read_csv('olist_orders_dataset.csv')

# customers
customers = pd.read_csv('olist_customers_dataset.csv')

# products
item = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
name = pd.read_csv('product_category_name_translation.csv')
pay = pd.read_csv('olist_order_payments_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')

# Geolocation
geolocation = pd.read_csv('olist_geolocation_dataset.csv')

In [ ]:
# data merge
rfm = orders.merge(customers, on="customer_id").merge(item, on="order_id").merge(products, on="product_id").merge(name, on="product_category_name").merge(pay, on="order_id").merge(sellers, on="seller_id").merge(reviews, on="order_id")

In [ ]:
# review 테이블의 score 열의 null 값을 0으로 채우기
rfm['review_score'].fillna(0, inplace=True)

## 이탈 고객의 주문 취소 비율 분석

#### R (Recency)
R은 '고객이 최근에 구매했는가?'를 의미

고객의 마지막 활동이 언제인지를 나타내는 변수로, 최근에 구매한 고객일수록 높은 점수가 부여

In [ ]:
recency = rfm.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
recency.rename(columns={'order_purchase_timestamp': 'recency'}, inplace=True)

In [ ]:
# order_purchase_timestamp 컬럼을 datetime 타입으로 변환
rfm['order_purchase_timestamp'] = pd.to_datetime(rfm['order_purchase_timestamp'])

# recency 컬럼의 데이터 타입을 datetime 타입으로 변환
recency['recency'] = pd.to_datetime(recency['recency'])

# 날짜 간의 차이를 계산
max_date = recency['recency'].max()
recency['diff_date'] = (max_date - recency['recency']).dt.days

In [ ]:
# diff_date 컬럼을 5등분하여 등급 부여
recency['r_score'] = pd.qcut(recency['diff_date'], q=5, labels=[5, 4, 3, 2, 1])

print(recency)

                     customer_unique_id             recency  diff_date r_score
0      0000366f3b9a7992bf8c76cfdf3221e2 2018-05-10 10:56:27        115       4
1      0000b849f77a49e4a4ce2b2a4ca5be3f 2018-05-07 11:11:27        118       4
2      0000f46a3911fa3c0805444483337064 2017-03-10 21:05:03        541       1
3      0000f6ccb0745a6a4b88665a16c9f078 2017-10-12 20:29:41        325       2
4      0004aac84e0df4da2b147fca70cf8255 2017-11-14 19:45:42        292       2
...                                 ...                 ...        ...     ...
93391  fffcf5a5ff07b0908bd4e2dbc735a684 2017-06-08 21:00:36        451       1
93392  fffea47cd6d3cc0a88bd621562a9d061 2017-12-10 20:07:56        266       3
93393  ffff371b4d645b6ecea244b27531430a 2017-02-07 15:49:16        572       1
93394  ffff5962728ec6157033ef9805bacc48 2018-05-02 15:17:41        123       4
93395  ffffd2657e2aad2907e67c3e9daecbeb 2017-05-02 20:18:45        488       1

[93396 rows x 4 columns]


In [ ]:
r_score_ranges = recency.groupby('r_score')['diff_date'].agg([min, max])
print(r_score_ranges)

         min  max
r_score          
5          0   95
4         96  180
3        181  272
2        273  387
1        388  728


#### F (Frequency)
F는 '고객이 얼마나 자주 구매했는가?'를 나타냄

고객이 얼마나 자주 구매했는지를 나타내는 변수로, 자주 구매한 고객일수록 높은 점수가 부여

In [ ]:
frequency = rfm.groupby('customer_unique_id')['order_purchase_timestamp'].nunique().reset_index()
frequency.rename(columns={'order_purchase_timestamp': 'frequency'}, inplace=True)

In [ ]:
def parse_values(x):
    switch = {
        1: 1,
        2: 2,
        3: 3,
        4: 4
    }
    return switch.get(x, 5)

frequency['f_score'] = frequency['frequency'].apply(parse_values)

print(frequency)

                     customer_unique_id  frequency  f_score
0      0000366f3b9a7992bf8c76cfdf3221e2          1        1
1      0000b849f77a49e4a4ce2b2a4ca5be3f          1        1
2      0000f46a3911fa3c0805444483337064          1        1
3      0000f6ccb0745a6a4b88665a16c9f078          1        1
4      0004aac84e0df4da2b147fca70cf8255          1        1
...                                 ...        ...      ...
93391  fffcf5a5ff07b0908bd4e2dbc735a684          1        1
93392  fffea47cd6d3cc0a88bd621562a9d061          1        1
93393  ffff371b4d645b6ecea244b27531430a          1        1
93394  ffff5962728ec6157033ef9805bacc48          1        1
93395  ffffd2657e2aad2907e67c3e9daecbeb          1        1

[93396 rows x 3 columns]


In [ ]:
# 각 등급의 범위 확인
f_score_ranges = frequency.groupby('f_score')['frequency'].agg([min, max])
print(f_score_ranges)

         min  max
f_score          
1          1    1
2          2    2
3          3    3
4          4    4
5          5   15


#### M (Monetary)
M은 '고객이 얼마나 구매했나?'를 측정하는 지표

고객이 구매한 총 금액을 의미하는 변수로, 구매 금액이 높은 고객일수록 높은 점수가 부여

In [ ]:
monetary = rfm.groupby('customer_unique_id')['payment_value'].sum().reset_index()
monetary.rename(columns={'payment_value': 'monetary'}, inplace=True)
rfm = rfm.merge(monetary, on='customer_unique_id')

In [ ]:
m_bins = pd.qcut(monetary['monetary'], 5, labels = [1, 2, 3, 4, 5])
monetary['m_score'] = m_bins

In [ ]:
# 각 등급의 범위 확인
m_score_ranges = monetary.groupby('m_score')['monetary'].agg([min, max])
print(m_score_ranges)

            min        max
m_score                   
1          9.59      56.08
2         56.09      89.88
3         89.89     140.32
4        140.33     239.32
5        239.34  109312.64


In [ ]:
# recency, frequency, monetary 데이터프레임 하나로 합치기
rfm_score = recency.merge(frequency, on = 'customer_unique_id')
rfm_score = rfm_score.merge(monetary, on = 'customer_unique_id')

# rfm_score을 합치기 위해 string으로 형 변환
rfm_score['r_score'] = rfm_score['r_score'].astype(str)
rfm_score['f_score'] = rfm_score['f_score'].astype(str)
rfm_score['m_score'] = rfm_score['m_score'].astype(str)

# rfm_score 합치기
rfm_score['rfm_score'] = rfm_score['r_score'] + rfm_score['f_score'] + rfm_score['m_score']

print(rfm_score)

                     customer_unique_id             recency  diff_date  \
0      0000366f3b9a7992bf8c76cfdf3221e2 2018-05-10 10:56:27        115   
1      0000b849f77a49e4a4ce2b2a4ca5be3f 2018-05-07 11:11:27        118   
2      0000f46a3911fa3c0805444483337064 2017-03-10 21:05:03        541   
3      0000f6ccb0745a6a4b88665a16c9f078 2017-10-12 20:29:41        325   
4      0004aac84e0df4da2b147fca70cf8255 2017-11-14 19:45:42        292   
...                                 ...                 ...        ...   
93391  fffcf5a5ff07b0908bd4e2dbc735a684 2017-06-08 21:00:36        451   
93392  fffea47cd6d3cc0a88bd621562a9d061 2017-12-10 20:07:56        266   
93393  ffff371b4d645b6ecea244b27531430a 2017-02-07 15:49:16        572   
93394  ffff5962728ec6157033ef9805bacc48 2018-05-02 15:17:41        123   
93395  ffffd2657e2aad2907e67c3e9daecbeb 2017-05-02 20:18:45        488   

      r_score  frequency f_score  monetary m_score rfm_score  
0           4          1       1    141.90      

In [ ]:
segments = {
    r'111|112|121|131|141|151' : '이탈',
    r'332|322|233|232|223|222|132|123|122|212|211' : '동면 ',
    r'155|154|144|214|215|115|114|113' : '놓치면 안 될',
    r'255|254|245|244|253|252|243|242|235|234|225|224|153|152|145|143|142|135|134|133|125|124' : '이탈 우려',
    r'331|321|312|221|213|231|241|251' : '휴면 예정',
    r'535|534|443|434|343|334|325|324' : '관심 필요',
    r'525|524|523|515|514|513|425|424|413|414|415|315|314|313' : '잠재',
    r'522|521|512|511|422|421|412|411|311' : '신규 고객',
    r'553|551|552|541|542|533|532|531|452|451|442|441|431|453|433|432|423|353|352|351|342|341|333|323' : '잠재 충성',
    r'543|444|435|355|354|345|344|335' : '충성',
    r'555|554|544|545|454|455|445' : 'VIP'
}

rfm_score['segment'] = rfm_score['rfm_score'].replace(segments, regex=True)

In [ ]:
conditions = [
    rfm_score["segment"] == "VIP",
    rfm_score["segment"] == "충성",
    rfm_score["segment"] == "잠재 충성",
    rfm_score["segment"] == "신규",
    rfm_score["segment"] == "잠재",
    rfm_score["segment"] == "관심 필요",
    rfm_score["segment"] == "휴면 예정",
    rfm_score["segment"] == "이탈 우려",
    rfm_score["segment"] == "놓치면 안 될",
    rfm_score["segment"] == "동면",
    rfm_score["segment"] == "이탈",
]
values = ["VIP", "VIP", "VIP", "GOLD", "GOLD", "GOLD", "SILVER", "SILVER", "SILVER", "BRONZE", "BRONZE"]

rfm_score["고객등급"] = np.select(conditions, values, default="")

In [ ]:
import plotly.express as px

# 고객 세그먼트별 고객 수 계산
segment_counts = rfm_score['segment'].value_counts().reset_index()
segment_counts.columns = ['segment', 'count']

# Bar 차트 생성
fig = px.bar(segment_counts, x='segment', y='count', title='고객 세그먼트 분포')

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': '고객 세그먼트 분포',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)


# 시각화 출력
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# 데이터를 고객 등급과 세그먼트로 그룹화하고 고객 수를 계산
customer_counts = rfm_score.groupby(['고객등급', 'segment']).size().reset_index(name='고객수')

# 고객 등급을 원하는 순서대로 정의
custom_order = ["VIP", "GOLD", "SILVER", "BRONZE"]
customer_counts['고객등급'] = pd.Categorical(customer_counts['고객등급'], categories=custom_order, ordered=True)

# 순서 기준으로 정렬
customer_counts = customer_counts.sort_values(by='고객등급')

# 시각화
fig = px.bar(customer_counts, x='고객등급', y='고객수', color='segment',
             title='고객 등급별 세그먼트별 고객 수')

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': '고객 등급별 세그먼트별 고객 수',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)

# 시각화 출력
fig.show()

In [ ]:
# 두 데이터프레임을 'customer_unique_id' 열을 기준으로 결합하고 결과를 rfm_score로 저장
rfm_score = pd.concat([rfm, rfm_score], axis=1)

In [ ]:
import plotly.express as px

# 데이터 프레임에서 취소 건수 확인
fig = px.bar(merged_data, x='고객등급', y='취소건수', color='segment',
             title='고객 등급별 취소건수',
             labels={'취소건수': 'Canceled count'},
             )

# x축 간격 설정
fig.update_xaxes(categoryorder="total ascending")

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': '고객 등급별 취소건수',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)


# 시각화 출력
fig.update_xaxes(tickangle=0)
fig.show()

In [ ]:
import plotly.express as px

# 데이터 프레임에서 취소 건수 확인
fig = px.bar(merged_data, x='segment', y='취소건수', color='고객등급',
             title='고객 등급별 취소건수',
             labels={'취소건수': 'Canceled count'},
             )

# x축 간격 설정
fig.update_xaxes(categoryorder="total ascending")

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': '고객 등급별 취소건수',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)


# 시각화 출력
fig.update_xaxes(tickangle=0)
fig.show()

In [ ]:
import plotly.express as px

# 취소 건수를 파이차트로 그리기
fig = px.pie(merged_data, values='취소건수', names='segment', color='고객등급',
             title='취소 건수')

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': '취소 건수',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)


# 시각화 출력
fig.show()


In [ ]:
# 주문 승인 시간이 NOT NULL이고 배송이 완료된 경우 필터링
completed_orders = rfm_score[(~rfm_score['order_approved_at'].isnull()) & (rfm_score['order_status'] == 'delivered')]

# 고객 등급과 세그먼트로 그룹화하고 구매 횟수 계산
frequency = completed_orders.groupby(['고객등급', 'segment'])['customer_unique_id'].count().reset_index()
frequency.rename(columns={'customer_unique_id': '빈도수'}, inplace=True)

# 결과 출력
print(frequency)

      고객등급     segment    빈도수    빈도수
0   BRONZE       동면 고객   7517   7517
1   BRONZE       이탈 고객   7513   7513
2     GOLD    관심 필요 고객    469    469
3     GOLD       신규 고객  18155  18155
4     GOLD       잠재 고객  32681  32681
5   SILVER  놓치면 안 될 고객  17219  17219
6   SILVER    이탈 우려 고객    692    692
7   SILVER    휴면 예정 고객   7138   7138
8      VIP      VIP 고객     29     29
9      VIP    잠재 충성 고객    142    142
10     VIP       충성 고객     55     55


In [ ]:

# 점수 계산 함수 정의
def calculate_score(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    else:
        return 5

# 각 점수 별 count 계산
score_counts = rfm_score.groupby(['고객등급', 'segment', 'f_score'])['customer_unique_id'].count().reset_index()
score_counts.rename(columns={'customer_unique_id': 'count'}, inplace=True)

# 결과 출력
print(score_counts)


      고객등급     segment f_score  count  count
0   BRONZE       동면 고객       1   7480   7480
1   BRONZE       동면 고객       2    180    180
2   BRONZE       동면 고객       3      6      6
3   BRONZE       이탈 고객       1   7658   7658
4   BRONZE       이탈 고객       2      7      7
5   BRONZE       이탈 고객       3      1      1
6     GOLD    관심 필요 고객       2    427    427
7     GOLD    관심 필요 고객       3     53     53
8     GOLD       신규 고객       1  18423  18423
9     GOLD       신규 고객       2     63     63
10    GOLD       잠재 고객       1  32433  32433
11    GOLD       잠재 고객       2    888    888
12  SILVER  놓치면 안 될 고객       1  17564  17564
13  SILVER    이탈 우려 고객       2    658    658
14  SILVER    이탈 우려 고객       3     41     41
15  SILVER    이탈 우려 고객       4      6      6
16  SILVER    이탈 우려 고객       5      1      1
17  SILVER    휴면 예정 고객       1   7271   7271
18  SILVER    휴면 예정 고객       2      6      6
19     VIP      VIP 고객       4     21     21
20     VIP      VIP 고객       5      9      9
21     VIP

In [ ]:
import plotly.express as px

# 'f_score' 컬럼을 숫자로 변환
score_counts['f_score'] = pd.to_numeric(score_counts['f_score'])

# 시각화
fig = px.bar(score_counts, x='segment', y='f_score', color='f_score',
             title='Frequency Score 분포',
             labels={'f_score': 'Score'},
             category_orders={'f_score': [1, 2, 3, 4, 5]}
             )

# 레이아웃 업데이트를 통해 title을 정중앙에 위치시킴
fig.update_layout(
    title={
        'text': 'Frequency Score 분포',
        'xref': 'paper',
        'yref': 'paper',
        'x': 0.5,
        'xanchor': 'center',
    },
    margin=dict(b=5)  # 아래 여백을 더 추가합니다.
)


# 시각화 출력
fig.show()
